This is notebook with WandBi sweep that can be performed with this project FNN implementation. 

In [1]:
#general libraries
import pandas as pd
import numpy as np
import copy
import wandb
#project code
from FNN import FNN
from OptimizersFunctions import *
from ActivFunctions import  *
from LossFunctions import *
from SuppFunctions import *
from TestingFunctions import *
from SweepFunctions import *

In [2]:
#specifying WandBi data for sweep
api_key = ""#specify your api_key, but never share it
entity = "DL_project_Group_70"
project = "CIFAR"

In [3]:
#loging to WandBi
wandb.login(api_key)

wandb: Currently logged in as: s253711 (DL_project_Group_70) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
#Specifying configuration for sweep
sweep_configuration = {
    "name": "sweepdemo",
    "method": "bayes",
    "metric": {"goal": "minimize", "name": "val_loss"},
    "parameters": {
        "dataset": {"values": ["CIFAR"]},#"MNIST","CIFAR"
        "epochs": {"values": [10,15,20]},
        "num_hidden_layers": {"values": [3, 4, 5]},
        "num_hidden_units": {"values": [ 64, 128, 256]},
        "optimizer": {"values": ["sgd","sgd_momentum","rmsprop","nag","adam"]},
        "learning_rate": {"min": 0.0001, "max": 0.01},
        "grad_clip": {"min": 0.5, "max": 5.0},
        "momentum": {"min": 0.8, "max": 0.95},#sgd_momentum,nag
        "beta": {"min": 0.9, "max": 0.99},#rmsprop
        "beta1": {"min": 0.8, "max": 0.95},#adam
        "beta2": {"min": 0.96, "max": 0.99},#adam
        "batch_size": {"values": [ 64, 128, 256]},
        "l_coeff": {"min": 1e-5, "max": 1e-3},
        "l_method": {"values": ["none","l1", "l2"]},
        "weights_init": {"values": ["RandomUni", "RandomNor", "XavUni", "XavNor", "HeUni", "HeNor"]},
        "activation_hidden": {"values": ["identity", "sigmoid", "tanh", "relu", "leaky_relu"]},
        "activation_output": {"values": ["sigmoid", "softmax"]},
        "loss_function": {"values": [ "CrossEntropy", "SoftmaxCrossEntropy"]},
    },
}

In [5]:
#setting sweep ID
sweep_id = wandb.sweep(sweep=sweep_configuration, project = project)

Create sweep with ID: xbv91wj5
Sweep URL: https://wandb.ai/DL_project_Group_70/CIFAR/sweeps/xbv91wj5


In [6]:
#starting sweep
wandb.agent(sweep_id, function=main,count=1)

wandb: Agent Starting Run: rgoy3udq with config:
wandb: 	activation_hidden: leaky_relu
wandb: 	activation_output: softmax
wandb: 	batch_size: 256
wandb: 	beta: 0.9710914580484324
wandb: 	beta1: 0.9426098035508972
wandb: 	beta2: 0.974204516695057
wandb: 	dataset: CIFAR
wandb: 	epochs: 15
wandb: 	grad_clip: 4.232214117718327
wandb: 	l_coeff: 0.0006867407820636122
wandb: 	l_method: l2
wandb: 	learning_rate: 0.003535180981468494
wandb: 	loss_function: SoftmaxCrossEntropy
wandb: 	momentum: 0.9058751897001248
wandb: 	num_hidden_layers: 3
wandb: 	num_hidden_units: 64
wandb: 	optimizer: adam
wandb: 	weights_init: HeNor


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▂▄▄▅▆▅▆▇▇▇▇▇██
train_loss,▆▅▆▃▅▃▁▄▇▅▅▃█▅▇
val_acc,▁▃▄▄▅▆▅▆▇▇▇▇▇██
val_loss,▅▄▆▃▄▃▁▄▆▅▅▃█▄▇
epoch,15
train_acc,0.48482
train_loss,163314.00663
val_acc,0.462
val_loss,33917.53142
